## Implementation of the Gale-Shapley Algorithm in Python

_Generates random preferences for men and women and confirms that the Gale-Shapley algorithm always returns a stable matching, providing a
clear explanation and validation of stability in the generated matchings._

In [26]:
women = ['Amanda', 'Barbara', 'Charlotte', 'Diana']
men = ['Adam', 'Bob', 'Charlie', 'David']

pref_dict_w = {
    'Amanda': ['Adam', 'Bob', 'Charlie', 'David'],
    'Barbara': ['Bob', 'David', 'Adam', 'Charlie'],
    'Charlotte': ['Charlie', 'Bob', 'Adam', 'David'],
    'Diana': ['David', 'Adam', 'Bob', 'Charlie'],
}

pref_dict_m = {
    'Adam': ['Amanda', 'Barbara', 'Charlotte', 'Diana'],
    'Bob': ['Amanda', 'Barbara', 'Charlotte', 'Diana'],
    'Charlie': ['Amanda', 'Barbara', 'Charlotte', 'Diana'],
    'David': ['Amanda', 'Barbara', 'Charlotte', 'Diana'],
}


In [27]:
def pref_to_rank(pref_list):
    return {
        a: {b: i for i, b in enumerate(a_pref)} for a, a_pref in pref_list.items()
    }

pref_rank_w = pref_to_rank(pref_dict_w)
pref_rank_m = pref_to_rank(pref_dict_m)

In [28]:
pref_dict_w, pref_dict_m


({'Amanda': ['Adam', 'Bob', 'Charlie', 'David'],
  'Barbara': ['Bob', 'David', 'Adam', 'Charlie'],
  'Charlotte': ['Charlie', 'Bob', 'Adam', 'David'],
  'Diana': ['David', 'Adam', 'Bob', 'Charlie']},
 {'Adam': ['Amanda', 'Barbara', 'Charlotte', 'Diana'],
  'Bob': ['Amanda', 'Barbara', 'Charlotte', 'Diana'],
  'Charlie': ['Amanda', 'Barbara', 'Charlotte', 'Diana'],
  'David': ['Amanda', 'Barbara', 'Charlotte', 'Diana']})

In [29]:
pref_rank_w, pref_rank_m

({'Amanda': {'Adam': 0, 'Bob': 1, 'Charlie': 2, 'David': 3},
  'Barbara': {'Bob': 0, 'David': 1, 'Adam': 2, 'Charlie': 3},
  'Charlotte': {'Charlie': 0, 'Bob': 1, 'Adam': 2, 'David': 3},
  'Diana': {'David': 0, 'Adam': 1, 'Bob': 2, 'Charlie': 3}},
 {'Adam': {'Amanda': 0, 'Barbara': 1, 'Charlotte': 2, 'Diana': 3},
  'Bob': {'Amanda': 0, 'Barbara': 1, 'Charlotte': 2, 'Diana': 3},
  'Charlie': {'Amanda': 0, 'Barbara': 1, 'Charlotte': 2, 'Diana': 3},
  'David': {'Amanda': 0, 'Barbara': 1, 'Charlotte': 2, 'Diana': 3}})

In [30]:
from collections import namedtuple

pair = namedtuple('pair', ['woman', 'man'])
pair('Amanda', 'Adam')

pair(woman='Amanda', man='Adam')

### Brute-Forcing It

In [23]:
from itertools import permutations

def stable_match_forced(
        *, women, men, pref_rank_w, pref_rank_m
):
    '''
    Solve stable matching problem using brute force

    women: set[str] set of women
    men: set[str] set of men
    pref_rank_w: dict[str, dict[str, int]] preference rank
    pref_rank_m: dict[str, dict[str, int]] preference rank
    '''

    w_rank = pref_to_rank(pref_rank_w)
    m_rank = pref_to_rank(pref_rank_m)

    w_seq = tuple(women)
    matching = (
        [
            pair(w, m)
            for w, m in zip(w_seq, m_seq)
        ]
        for m_seq in permutations(men)
    )

    for match in matching:
        match_w = {pair.woman: pair for pair in match}
        match_m = {pair.man: pair for pair in match}

        unstable = any(
            (
                w_rank[w][m] < w_rank[w][match_w[w].man] and
                m_rank[m][w] < m_rank[m][match_m[m].woman]
            )
            for w in women
            for m in men
            if w != match_m[m].woman
            if m != match_w[w].man
        )
        if not unstable:
            return match

In [24]:
stable_match_forced(
    women = {'Amanda', 'Barbara', 'Charlotte', 'Diana'},
    men = {'Adam', 'Bob', 'Charlie', 'David'},
    pref_rank_w = {
        'Amanda': ['Adam', 'Bob', 'Charlie', 'David'],
        'Barbara': ['Bob', 'David', 'Adam', 'Charlie'],
        'Charlotte': ['Charlie', 'Bob', 'Adam', 'David'],
        'Diana': ['David', 'Adam', 'Bob', 'Charlie'],
    },
    pref_rank_m = {
        'Adam': ['Amanda', 'Barbara', 'Charlotte', 'Diana'],
        'Bob': ['Amanda', 'Barbara', 'Charlotte', 'Diana'],
        'Charlie': ['Amanda', 'Barbara', 'Charlotte', 'Diana'],
        'David': ['Amanda', 'Barbara', 'Charlotte', 'Diana'],
    }
)

[pair(woman='Diana', man='David'),
 pair(woman='Barbara', man='Bob'),
 pair(woman='Charlotte', man='Charlie'),
 pair(woman='Amanda', man='Adam')]

In [25]:
%timeit stable_match_forced(women={'Amanda', 'Barbara', 'Charlotte', 'Diana'}, men={'Adam', 'Bob', 'Charlie', 'David'}, pref_rank_w=pref_rank_w, pref_rank_m=pref_rank_m)

24.6 µs ± 197 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Gale-Shapley

In [31]:
from collections import deque

def gale_shapley(*, A, B, A_pref, B_pref):
    '''
    Solve stable matching problem using Gale-Shapley algorithm

    A: set[str] set of A
    B: set[str] set of B
    A_pref: dict[str, list[str]] preference list of A
    B_pref: dict[str, list[str]] preference list of B
    
    return: dict[str, str] matching
    '''

    B_rank = pref_to_rank(B_pref)
    ask_list = {a: deque(b_list) for a, b_list in A_pref.items()}
    match = {}

    a_rem = set(A)
    while len(a_rem) > 0:
        a = a_rem.pop()
        b = ask_list[a].popleft()
        if b not in match:
            match[b] = a
        else:
            a_ = match[b]
            b_pref_a_ = B_rank[b][a_] < B_rank[b][a]
            if b_pref_a_:
                a_rem.add(a)
            else:
                a_ = match[b] 
                b_pref_a_ = B_rank[b][a_] < B_rank[b][a]
                if b_pref_a_:
                    a_rem.add(a)
                else:
                    a_rem.add(a_)
                    match[b] = a

    return [(a, b) for b, a in match.items()]


In [32]:
gale_shapley(
    A={'Amanda', 'Barbara', 'Charlotte', 'Diana'},
    B={'Adam', 'Bob', 'Charlie', 'David'},
    A_pref={
        'Amanda': ['Adam', 'Bob', 'Charlie', 'David'],
        'Barbara': ['Bob', 'David', 'Adam', 'Charlie'],
        'Charlotte': ['Charlie', 'Bob', 'Adam', 'David'],
        'Diana': ['David', 'Adam', 'Bob', 'Charlie'],
    },
    B_pref={
        'Adam': ['Amanda', 'Barbara', 'Charlotte', 'Diana'],
        'Bob': ['Amanda', 'Barbara', 'Charlotte', 'Diana'],
        'Charlie': ['Amanda', 'Barbara', 'Charlotte', 'Diana'],
        'David': ['Amanda', 'Barbara', 'Charlotte', 'Diana'],
    }
)

[('Diana', 'David'),
 ('Barbara', 'Bob'),
 ('Charlotte', 'Charlie'),
 ('Amanda', 'Adam')]

In [33]:
%timeit gale_shapley(A={'Amanda', 'Barbara', 'Charlotte', 'Diana'}, B={'Adam', 'Bob', 'Charlie', 'David'}, A_pref=pref_dict_w, B_pref=pref_dict_m)

3.99 µs ± 23.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Validating Stability

In [34]:
def validate_stability(match, pref_rank_w, pref_rank_m):
    '''
    Validate the stability of a matching

    match: list of pairs representing the matching
    pref_rank_w: dict[str, dict[str, int]] preference rank of women
    pref_rank_m: dict[str, dict[str, int]] preference rank of men

    return: bool indicating whether the matching is stable or not
    '''

    for pair in match:
        woman, man = pair

        # Check if there is a man that the woman prefers over her current match
        for m in pref_rank_w[woman]:
            if m == man:
                break
            if pref_rank_w[woman][m] < pref_rank_w[woman][man]:
                # Check if the man prefers the woman over his current match
                if pref_rank_m[man][woman] < pref_rank_m[man][match[man]]:
                    return False

    return True


In [37]:
# check if the matching is stable
match = gale_shapley(A={'Amanda', 'Barbara', 'Charlotte', 'Diana'}, B={'Adam', 'Bob', 'Charlie', 'David'}, A_pref=pref_dict_w, B_pref=pref_dict_m)
match_bf = stable_match_forced(women={'Amanda', 'Barbara', 'Charlotte', 'Diana'}, men={'Adam', 'Bob', 'Charlie', 'David'}, pref_rank_w=pref_dict_w, pref_rank_m=pref_dict_m)
print(validate_stability(match, pref_rank_w, pref_rank_m))
print(validate_stability(match_bf, pref_rank_w, pref_rank_m))

True
True


## Randomizing Pairs

In [41]:
# Create a random list of women from input size
import random
import time
import matplotlib.pyplot as plt

def gen_prefs(n):
    return [random.sample(range(n), n) for _ in range(n)]

def gen_prefs_dict(n):
    return {i: gen_prefs(n) for i in range(n)}